# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-V3-Base',
 '/deepseek-ai/DeepSeek-V3',
 '/Qwen/QVQ-72B-Preview',
 '/answerdotai/ModernBERT-base',
 '/black-forest-labs/FLUX.1-dev',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/Qwen/QVQ-72B-preview',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces',
 '/datasets/HuggingFaceTB/finemath',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/agibot-world/AgiBotWorld-Alpha',
 '/datasets/amphora/QwQ-LongCoT-130K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-V3-Base
Updated
1 day ago
•
4.67k
•
949
deepsee

In [21]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-Base\nUpdated\n1 day ago\n•\n4.67k\n•\n949\ndeepseek-ai/DeepSeek-V3\nUpdated\n1 day ago\n•\n17.5k\n•\n743\nQwen/QVQ-72B-Preview\nUpdated\n6 days ago\n•\n28.3k\n•\n407\nanswerdotai/ModernBERT-base\nUpdated\n5 days ago\n•\n46.9k\n•\n542\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.18M\n•\n7.61k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n2.33k\n🏢\nTRELLIS\nScalable and Versatile 3D Generation from 

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'documentation', 'url': 'https://huggingface.com/docs'}]}


```markdown
# Hugging Face Brochure

## Welcome to Hugging Face

### The AI Community Building the Future

At Hugging Face, we are a vibrant community dedicated to the advancement of machine learning. Our platform facilitates collaboration on models, datasets, and applications—making it the go-to destination for AI enthusiasts, researchers, and organizations alike.

---

## Our Offerings

- **Models**: Access over 400,000 machine learning models contributed by the community.
- **Datasets**: Browse through more than 100,000 datasets for various AI applications.
- **Spaces**: Discover innovative applications running on our platform.
- **Enterprise Solutions**: Tailored services for over 50,000 organizations, offering enterprise-grade security, access controls, and dedicated support.

### Popular Models This Week

- **DeepSeek-V3-Base**: 4.67k updates
- **ModernBERT-base**: 46.9k updates
- **FLUX.1-dev**: 1.18M updates

---

## Company Culture

At Hugging Face, we are on a mission to **democratize good machine learning**, fostering an environment rooted in collaboration, transparency, and innovation. Our team thrives on diverse ideas and a shared passion for making machine learning accessible to all.

**Team Size**: 223 committed professionals collaborating dynamically.

**Core Values**:
- **Community-Driven**: We believe in the power of open-source collaboration.
- **Innovative Spirit**: Constantly learning and adapting to push the boundaries of what is possible in AI.
- **Inclusivity**: We embrace a culture where every voice is valued, creating a supportive and productive workplace.

---

## Customers

Join thousands of organizations including industry leaders like:

- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Meta**

We serve various sectors, from startups to large enterprises, all aiming to harness the potential of machine learning for growth and innovation.

---

## Careers at Hugging Face

We are always on the lookout for passionate talent! Whether you are a software engineer, data scientist, or a creative thinker, we invite you to explore opportunities with us.

#### Why Work with Us?
- **Innovative Projects**: Work with cutting-edge technology in machine learning.
- **Collaborative Environment**: Engage with a diverse team from around the globe.
- **Professional Growth**: We invest in your development and learning.

### Join Us!
Explore job openings at [Hugging Face Careers](https://huggingface.co/jobs).

---

## Get In Touch

To learn more about Hugging Face, our products, or partnership opportunities, visit our website or follow us on social media:

- **Website**: [Hugging Face](https://huggingface.co)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face](https://www.linkedin.com/company/huggingface)

---

Together, let's build the future of AI!
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## **About Us**
Hugging Face is more than just a technology company; we are a vibrant community dedicated to building the future of artificial intelligence. Our platform serves as a hub where the machine learning community comes together to collaborate on models, datasets, and applications, advancing the field of AI for everyone.

## **What We Offer**
- **Models:** Access to over 400,000 machine learning models, including the latest updates from our vibrant community.
- **Datasets:** Explore and contribute to a rich library of over 100,000 datasets, aimed at powering your AI projects.
- **Spaces:** A dedicated environment where you can deploy and share applications with ease.
- **Enterprise Solutions:** Specialized tools for businesses that require advanced capabilities, security, and support with plans starting at $20/user/month.

## **Our Community**
Join a thriving ecosystem with more than 50,000 organizations, including industry giants like:
- Google
- Amazon Web Services
- Microsoft
- Intel

Be part of a collaborative environment that values innovation and accessibility in machine learning.

## **Company Culture**
At Hugging Face, we foster a culture of openness, collaboration, and creativity. Our values include:
- **Community-Centric:** We prioritize the collaboration of the global AI community to enable everyone to benefit from advancements in technology.
- **Diversity & Inclusion:** We believe that diverse teams create better solutions and therefore encourage participation from all backgrounds.
- **Growth Mindset:** Continuous learning and development are at the heart of our company ethos, with opportunities for all team members to expand their skills.

## **Careers at Hugging Face**
We are always on the lookout for passionate individuals to join our team! Opportunities abound in various roles such as:
- Machine Learning Engineers
- Data Scientists
- Product Managers
- Community Managers

Working with us means contributing to cutting-edge AI technology while collaborating with some of the brightest minds in the field.

## **Join Us!**
Whether you're a customer, an investor, or a potential recruit, we invite you to explore what Hugging Face has to offer. Together, we can push the boundaries of artificial intelligence!

**Website:** [Hugging Face](https://huggingface.co)  
**Social Media:** Follow us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/hugging-face)

Let’s build the future of AI, together!

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🤗 HuggingFace Brochure: Join Us for a Hug-tastic AI Adventure! 🚀

---

### **Welcome to Hugging Face!**  
Where AI Models Meet Community Love 💖

At Hugging Face, we believe in a world where artificial intelligence hugs you back! We’re not just “another tech company” — we’re a thriving AI community on a *hugging* mission to build the future one model at a time. Our platform celebrates collaboration, innovation, and yes, tons of friendly byte-sized fun!

---

## **What’s Hot This Week? 🔥**
- **DeepSeek-V3-Base**: Hotter than a fresh batch of cookies! 🍪 
- **QVQ-72B-Preview**: Because who needs sleep when you can preview cutting-edge AI models?
- **FLUX.1-dev**: It’s like the cool kid everyone wants to be friends with — 1.18M followers and counting!

(Just so you know, we’ve got over **400k models** and **100k datasets** fresh out of our hug factory! 😄)

---

## **Spaces & Places**  
Feeling adventurous? Explore our *Spaces*! 🌌
- **TRELLIS**: 3D magic — no wand required! 🪄
- **Kolors Virtual Try-On**: Because trying on outfits is so last season. 

And if you still can’t find the perfect fit, don't worry! We have **150k+ applications** to explore. 

---

## **Our Values: Collaboration, Curiosity & Cuddles!** 🐻  
At Hugging Face, we work collaboratively (and sometimes with a side of cuddling). Here's what you can expect:
- **Community-driven model-building** that even grandma would approve of! 👵
- **Open-source** culture because why not share the love?
- The ability to “hug” models, datasets, and applications seamlessly – it's all within your reach!

---

## **Who Do We Work With?**  
More than **50,000 organizations** and their happy little models! Here are a few cool cats on the team:
- **Meta**: Keeping it social since the dawn of AI.
- **Amazon**: Probably figuring out how to improve the Prime delivery using AI right now. 📦🤖
- **Google**: The tech giant that couldn't resist our cuddly charm! 

---

## **Careers: Join the Hug Crew! 👩‍💻👨‍💻**  
Think you’re a good fit for our quirky family? We’re on the lookout for talented and kind-hearted folks who want to dive into the world of machine learning with joy! Why? Because who wouldn’t want to hug data for a living? 

- Perks include an environment where no question is too silly and your AI dreams can run wild (and free)!
- Positions available in engineering, data science, community management, and more. 

*Let’s build a hug-friendly workplace together!*

---

## **Why Hugging Face?**  
- Because “Hugging” is what we do best… and it’s way more fun than “Shooting” or “Nudging”! 😄
- Because breaking barriers in AI shouldn’t feel like a lonely journey!
- Because here, friends don’t let friends work alone.

So, whether you’re looking to invest in some AI magic, find a job that feels like a warm hug from grandma, or simply want to explore the coolest AI community on the block, Hugging Face is where you wanna be! 🤗

---

## **Let’s Hug It Out!**  
Join us today and help build a future where AI is not just smart, but also friendly! 

- 🌐 [Website](https://www.huggingface.co)
- 💌 [Contact Us](mailto:support@huggingface.co)

Hugging Face: Where AI and hugs come together in perfect harmony! 🎤💥

---

Feel free to print, share, or at the very least, give yourself a hug while you dream of joining the Hugging Face family!  
**Because who doesn’t need more hugs in their life?** 🥳

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>